# Populating Neo4j Graph Database

In [146]:
import os
import json
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

NEO4J_URL = os.getenv("NEO4J_CONNECTION_URL")
NEO4J_USERNAME = os.getenv("NEO4J_USER")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# https://api.python.langchain.com/en/latest/graphs/langchain_community.graphs.neo4j_graph.Neo4jGraph.html
graph = Neo4jGraph(
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD
)


def wipe_graph(graph=graph):
    graph.query(query="MATCH (n) DETACH DELETE n")
    graph.query(query="CALL apoc.schema.assert({},{},true) YIELD label, key RETURN *")
    graph.refresh_schema()
    print(graph.get_structured_schema)

In [147]:
wipe_graph()

{'node_props': {}, 'rel_props': {}, 'relationships': [], 'metadata': {'constraint': [], 'index': []}}


#### Reading database and table metadata.

In [148]:
with open("metadata.json", encoding='utf-8') as file:
    metadata = json.load(file)

#### Creating uniqueness constraint for expected entities

In [149]:
query_string = """
CREATE CONSTRAINT table_uniqueness_rule IF NOT EXISTS 
    FOR (tb:Table) REQUIRE tb.table_full_name IS UNIQUE
"""

graph.query(query=query_string)

query_string = """
CREATE CONSTRAINT column_uniqueness_rule IF NOT EXISTS 
    FOR (col:Column) REQUIRE col.column_name IS UNIQUE
"""

graph.query(query=query_string)

query_string = """
CREATE CONSTRAINT database_uniqueness_rule IF NOT EXISTS
    FOR (db:Database) REQUIRE db.database_name IS UNIQUE
"""

graph.query(query=query_string)

[]

#### Creating TABLE entities

In [150]:
for table in metadata['table_metadata']:

    params = {
        "table_name": table['table_name'],
        "database_name": table['database_name'],
        "table_full_name": '.'.join([table['database_name'], table['table_name']]),
        "table_logic_name": table['table_logic_name'],
        "table_description": table['table_description'],
        "table_columns": [column['column_name'] for column in table['table_columns']],
        "table_primary_key": table['table_primary_key']
    }
    query_string = """
    CREATE (tb:Table {
        table_name: $table_name,
        table_logic_name: $table_logic_name,
        database_name: $database_name,
        table_full_name: $table_full_name,
        table_description: $table_description,
        table_columns: $table_columns,
        table_primary_key: $table_primary_key
        }
    )
    """

    graph.query(query=query_string, params=params)

    print(f"Entity {table['table_name']} created successfully...")

Entity fact_orders created successfully...
Entity dim_order_items created successfully...
Entity dim_sellers created successfully...
Entity dim_products created successfully...
Entity dim_order_reviews created successfully...
Entity dim_order_payments created successfully...
Entity dim_geolocation created successfully...
Entity dim_customers created successfully...


#### Creating DATABASE entities

In [151]:
for database in metadata['database_metadata']:

    for table in metadata['table_metadata']:

        params = {
            "database_name": database['database_name'],
            "database_description": database['database_description'],
            "table_name": table['table_name']
        }

        query_string = """
        MERGE (db:Database {database_name: $database_name})
        ON CREATE SET // On first execution creates the database entity
            db.database_description = $database_description,
            db.database_tables = [$table_name]
        ON MATCH SET // Updates table list in database entity if a database with the same name exists
            db.database_tables = db.database_tables + [$table_name]
        """

        graph.query(query=query_string, params=params)

#### Creating relationship CONTAINS between Database and Table entities

In [152]:
query_string = """
    MATCH (tb:Table)
    MATCH (db:Database)
    WHERE tb.table_name in db.database_tables
    CREATE (db)-[:CONTAINS]->(tb)
"""

graph.query(query=query_string)

[]

#### Creating Column entities

In [153]:
for table in metadata['table_metadata']:
    for column in table['table_columns']:
        params = {
            "table_name": table['table_name'],
            "column_name": column['column_name'],
            "column_description": column['column_description'],
            "data_type": column['data_type']
        }

        query_string = """
        MATCH (tb:Table) WHERE tb.table_name = $table_name 
        MERGE (col:Column {column_name: $column_name})
        ON CREATE SET
            col.column_description = $column_description,
            col.data_type = $data_type,
            col.located_at = [$table_name],
            col.is_primary_key_at = [],
            col.is_foreign_key_at = []
        ON MATCH SET
            col.located_at = col.located_at + [tb.table_name]
        """

        graph.query(query=query_string, params=params)

#### Creating HAS_COLUMN relationship between Table and Column entities

In [154]:
query_string = """ 
    MATCH (tb:Table)
    MATCH (col:Column)
    WHERE tb.table_name in col.located_at
    CREATE (tb)-[:HAS_COLUMN]->(col)
"""

graph.query(query=query_string)

[]

#### Creating IS_PRIMARY_KEY relationship between Column and Table entities

In [155]:
query_string = """ 
    MATCH (tb:Table)
    MATCH (col:Column) WHERE col.column_name in tb.table_primary_key
    MERGE (col)-[pk:IS_PRIMARY_KEY]->(tb)
"""

graph.query(query=query_string)

[]

#### Creating _is_primary_key_at_ property in all Column entities

In [156]:
query_string = """ 
    MATCH (tb:Table)
    MATCH (col:Column) WHERE col.column_name in tb.table_primary_key
    SET col.is_primary_key_at = col.is_primary_key_at + tb.table_name
"""

graph.query(query=query_string)

[]

#### Creating _is_foreign_key_at_ property in all Column entities

In [157]:
query_string = """
MATCH (tb:Table)
MATCH (col:Column)
WHERE (col)-[:IS_PRIMARY_KEY]->(tb)
SET col.is_foreign_key_at = apoc.coll.disjunction(col.located_at, [tb.table_name])
"""

graph.query(query=query_string)

[]

#### Creating IS_FOREIGN_KEY relationship between Column and Tables entities 

In [158]:
query_string = """ 
    MATCH (tb:Table)
    MATCH (col:Column) WHERE tb.table_name in col.is_foreign_key_at
    MERGE (col)-[:IS_FOREIGN_KEY]->(tb)
"""

graph.query(query=query_string)

[]

### Creating Vector Index

In [159]:
graph.query("""
         CREATE VECTOR INDEX `database_embeddings` IF NOT EXISTS
          FOR (db:Database) ON (db.text_embeddings) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

graph.query("""
         CREATE VECTOR INDEX `table_embeddings` IF NOT EXISTS
          FOR (tb:Table) ON (tb.text_embeddings) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

graph.query("""
         CREATE VECTOR INDEX `columns_embeddings` IF NOT EXISTS
          FOR (col:Column) ON (col.text_embeddings) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

In [165]:
graph.refresh_schema()

print(graph.get_schema)

Node properties:
Database {database_description: STRING, database_name: STRING, database_tables: LIST}
Table {table_logic_name: STRING, table_name: STRING, table_columns: LIST, table_primary_key: LIST, database_name: STRING, table_full_name: STRING, table_description: STRING}
Column {data_type: STRING, is_primary_key_at: LIST, is_foreign_key_at: LIST, column_name: STRING, column_description: STRING, located_at: LIST}
Relationship properties:

The relationships:
(:Database)-[:CONTAINS]->(:Table)
(:Table)-[:HAS_COLUMN]->(:Column)
(:Column)-[:IS_FOREIGN_KEY]->(:Table)
(:Column)-[:IS_PRIMARY_KEY]->(:Table)


#### Creating _text_embedding_ property in all entities and populate them with Embeddings Vector

We'll use those later to execute similarity search

In [167]:
database_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="database_embeddings",
    node_label="Database",
    text_node_properties=["database_name", "database_description"],
    embedding_node_property="text_embeddings"
)

table_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="table_embeddings",
    node_label="Table",
    text_node_properties=["table_name","table_logic_name", "table_description"],
    embedding_node_property="text_embeddings"
)

column_vector = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    url=NEO4J_URL,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="column_embeddings",
    node_label="Column",
    text_node_properties=["column_name","column_logic_name", "column_description"],
    embedding_node_property="text_embeddings"
)

In [168]:
table_vector.similarity_search("Quais os produtos mais vendidos em 2018?",k=8)

[Document(page_content='\ntable_name: dim_products\ntable_logic_name: Tabela dimensão de produtos\ntable_description: A tabela `dim_products` representa a dimensão produtos no e-commerce Olist. Nela são catalogados todos os produtos da plataforma do e-commerce assim como informações sobre eles como a categoria dos produtos, o peso, largura, comprimento, altura, comprimento da descrição do produto e quantidade de fotos disponíveis.', metadata={'database_name': 'olist', 'table_full_name': 'olist.dim_products', 'table_primary_key': ['product_id'], 'table_columns': ['product_id', 'product_category_name', 'product_name_length', 'product_description_length', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']}),
 Document(page_content='\ntable_name: dim_sellers\ntable_logic_name: Tabela dimensão de vendedores\ntable_description: A tabela `dim_sellers` contém todos os vendedores da plataforma. A tabela contem identificação do vendedor, seu p

In [86]:
schema_description = graph.query(
    query="CALL apoc.ml.schema({apiKey: $openai_api_key, model: 'gpt-4o'}) yield value RETURN *",
    params={"openai_api_key": OPENAI_API_KEY}
    )

In [96]:
print(schema_description[0]["value"])

The schema represents a database catalog structure, where:
1. A `:Database` node represents a specific database.
2. A `:Table` node represents tables within the database.
3. The relationship `:CONTAINS` links `:Database` to its `:Table` nodes.
4. The relationship `:HAS_COLUMN` links a `:Table` node to its columns.
5. Each `:Table` includes metadata such as logic name, primary key, and description.
